# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-5356f64747-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Ancient Rome explorative search") 

Consider the following exploratory information need:

> The goal of this workflow is to study the Ancient Rome and some of the main events connected to that historical period. For instance, the emperors, killings, wars and so on.

## Useful URIs for the current workflow
The following data are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q1048`| Julius Caesar       | node |





Also consider that

```
wd:Q1048 ?p ?obj .
```

is the BGP to retrieve all **properties of Julius Caesar**.

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for Ancient Rome

2. Find all the Roman Dictators (return the IRI and the name of the dictator)

3. Identify the BGP for Roman Empire and Roman Republic

4. Return the IRI and the name of all Roman emperors killed by homicide (see "manner of death") and, for each killed emperor, return also the IRI and name of the killer, if present in Wikidata.

5. Return the Roman emperors who killed another Roman emperor (the result set must a list of 4-tuples with killer IRI and label, and killed IRI and label).

6. Find the BGP for war

7. Find the wars (note that "war" and "battle" are not the same thing) participated by the Roman Empire (return IRI and name of the war).

8. Consider all the wars participated by the Roman Empire. For each war return the number of related battles and the opponent (a war is composed of one or more battles) (the result set must be a list of 5-tuples war IRI, label, opponent IRI and label, and #battles).


## Task 1
Identify the BGP for Ancient Rome

In [6]:
# write your final query
# getting the BGP of Ancient Rome data via Julius Caesar

queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q1048 wdt:P27 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1747689'), ('name', 'Ancient Rome')]
1


Final query for this task

In [10]:
# write your final query
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1747689 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('name', 'population')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1282'), ('name', 'OpenStreetMap tag or key')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1667'), ('name', 'Getty Thesaurus of Geographic Names ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1792'), ('name', 'category of associated people')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'im

## Task 2
Find all the Roman Dictators (return the IRI and the name of the dictator)

In [14]:
# getting the roman dictators data via Julius Caesar
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1048 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3701'), ('name', 'incarnation of')]
[('p', 'http://www.wikidata.org/prop/direct/P1003'), ('name', 'National Library of Romania ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('name', 'Vatican Library ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('name', 'member of political party')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1035'), ('name', 'honorific suffix')]
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('name', 'relative')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/p

Final query for this task

In [13]:
# write your final query
queryString = """
SELECT DISTINCT ?person ?name
WHERE {
   # bind something
   ?person wdt:P39 wd:Q236885 .
   # get the label
   ?person sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q155312'), ('name', 'Gaius Claudius Centho')]
[('person', 'http://www.wikidata.org/entity/Q297783'), ('name', 'Appius Claudius Caecus')]
[('person', 'http://www.wikidata.org/entity/Q703448'), ('name', 'Marcus Livius Salinator')]
[('person', 'http://www.wikidata.org/entity/Q529867'), ('name', 'Titus Lartius')]
[('person', 'http://www.wikidata.org/entity/Q15034789'), ('name', 'Manius Valerius Maximus')]
[('person', 'http://www.wikidata.org/entity/Q359810'), ('name', 'Lucius Caecilius Metellus')]
[('person', 'http://www.wikidata.org/entity/Q929498'), ('name', 'Quintus Caecilius Metellus')]
[('person', 'http://www.wikidata.org/entity/Q519186'), ('name', 'Quintus Hortensius')]
[('person', 'http://www.wikidata.org/entity/Q637596'), ('name', 'Quintus Servilius Priscus Structus Fidenas')]
[('person', 'http://www.wikidata.org/entity/Q1223522'), ('name', 'Caius Poetelius Libo Visolus')]
[('person', 'http://www.wikidata.org/entity/Q682087'), ('n

## Task 3
Identify the BGP for Roman Empire and Roman Republic

In [39]:
# getting Roman Empire and Roman Republic data via Julius Ceasar- country of citizenship relation to country of citizenship-has parts relationship 
queryString = """
SELECT DISTINCT ?p ?name ?region
WHERE {
   # bind something
   wd:Q1048 wdt:P27 ?obj .
   ?obj wdt:P527 ?region.
  
   # get the label
   ?region sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)





Results
[('name', 'Roman Empire'), ('region', 'http://www.wikidata.org/entity/Q2277')]
[('name', 'Byzantine Empire'), ('region', 'http://www.wikidata.org/entity/Q12544')]
[('name', 'Roman Republic'), ('region', 'http://www.wikidata.org/entity/Q17167')]
[('name', 'Roman Kingdom'), ('region', 'http://www.wikidata.org/entity/Q201038')]
4


Final query for this task

In [37]:
# write your final query
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q2277 ?p ?obj .
   wd:Q17167 ?p ?obj.
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P140'), ('name', 'religion')]
[('p', 'http://www.wikidata.org/prop/direct/P2348'), ('name', 'time period')]
[('p', 'http://www.wikidata.org/prop/direct/P2936'), ('name', 'language used')]
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('name', 'continent')]
[('p', 'http://www.wikidata.org/prop/direct/P3075'), ('name', 'official religion')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P36'), ('name', 'capital')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P38'), ('name', 'currency')]
[('p', 'http://www.wikidata.org/prop/direct/P41'), ('name', 'flag image')]
10


## Task 4
Return the IRI and the name of all Roman emperors killed by homicide (see "manner of death") and, for each killed emperor, return also the IRI and name of the killer, if present in Wikidata.

In [56]:
#getting to homicide and killer data by "manner of death" and "killed by" relation of Julius Caesar.
queryString = """
SELECT DISTINCT ?deathcause ?killer ?name 
WHERE {
   # bind something
   wd:Q1048 wdt:P1196 ?deathcause .
   #wd:Q1048 wdt:P157 ?killer. USED TO GET KILLER DATA
  
   # get the label
   ?deathcause sc:name ?name.
   #?killer sc:name ?name. USED TO GET KILLER DATA
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)





Results
[('deathcause', 'http://www.wikidata.org/entity/Q149086'), ('name', 'homicide')]
1


Final query for this task

In [75]:
# write your final query
queryString = """
SELECT DISTINCT ?person ?killer ?nameKiller ?nameEmperor 
WHERE {
   # bind something
   ?person wdt:P1196 wd:Q149086 . #data of emperors killed by homice
   ?person wdt:P157 ?killer. 
   # get the label
   ?person sc:name ?nameEmperor.
   ?killer sc:name ?nameKiller.
}limit 20
"""

print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q13645258'), ('killer', 'http://www.wikidata.org/entity/Q3530369'), ('nameKiller', 'Toghon Taishi'), ('nameEmperor', 'Esekü')]
[('person', 'http://www.wikidata.org/entity/Q24929787'), ('killer', 'http://www.wikidata.org/entity/Q7649086'), ('nameKiller', 'Susima'), ('nameEmperor', 'Dharma')]
[('person', 'http://www.wikidata.org/entity/Q107191433'), ('killer', 'http://www.wikidata.org/entity/Q107340634'), ('nameKiller', 'Zhang Sheng'), ('nameEmperor', 'Chen Jingji')]
[('person', 'http://www.wikidata.org/entity/Q107199626'), ('killer', 'http://www.wikidata.org/entity/Q107199590'), ('nameKiller', 'Wu Song'), ('nameEmperor', 'Granny Wang')]
[('person', 'http://www.wikidata.org/entity/Q107291928'), ('killer', 'http://www.wikidata.org/entity/Q107199590'), ('nameKiller', 'Wu Song'), ('nameEmperor', 'Li Waizhuan')]
[('person', 'http://www.wikidata.org/entity/Q1496'), ('killer', 'http://www.wikidata.org/entity/Q216275'), ('nameKiller', 'Lapulap

## Task 5
Return the Roman emperors who killed another Roman emperor (the result set must a list of 4-tuples with killer IRI and label, and killed IRI and label).

In [97]:
queryString = """
SELECT DISTINCT ?sub ?name
WHERE {
   # bind something
   #wd:Q1048 ?p ?obj . #getting properties of Julius Caesar
   #wd:Q1048 wdt:P39 ?obj. #by Caesar-"position held" relation getting "Roman dictator" ?p ?obj." data
   #wd:Q236885 ?p ?obj. #seeing "Roman dictator" data
   #wd:Q236885 wdt:P17 ?obj. #by using "Roman dictator"-"country" relationship we get to 'Ancient Rome' data
   #wd:Q1747689 ?p ?obj. #seeing "Ancient Rome" data
   #wd:Q1747689 wdt:P527 ?obj. #by using "Ancient Rome"-"has parts" relationship we get to 'Roman Republic' data
   #wd:Q2277 ?p ?obj. #seeing "Roman Republic" data and we get to "roman emperor" data.
   ?sub ?p wd:Q842606. #we get all the roman emperors.
   # get the label
   ?sub sc:name ?name.
}limit 20
"""

print("Results")
x=run_query(queryString)

Results
[('sub', 'http://www.wikidata.org/entity/Q46780'), ('name', 'Aurelian')]
[('sub', 'http://www.wikidata.org/entity/Q159798'), ('name', 'Honorius')]
[('sub', 'http://www.wikidata.org/entity/Q437604'), ('name', 'Julius Saturninus')]
[('sub', 'http://www.wikidata.org/entity/Q211396'), ('name', 'Magnus Maximus')]
[('sub', 'http://www.wikidata.org/entity/Q550622'), ('name', 'Maximus of Hispania')]
[('sub', 'http://www.wikidata.org/entity/Q1777'), ('name', 'Maximinus Thrax')]
[('sub', 'http://www.wikidata.org/entity/Q518890'), ('name', 'Gaius Julius Verus Maximus')]
[('sub', 'http://www.wikidata.org/entity/Q172471'), ('name', 'Valens')]
[('sub', 'http://www.wikidata.org/entity/Q314828'), ('name', 'Vetranio')]
[('sub', 'http://www.wikidata.org/entity/Q187489'), ('name', 'Silvanus')]
[('sub', 'http://www.wikidata.org/entity/Q1427'), ('name', 'Hadrian')]
[('sub', 'http://www.wikidata.org/entity/Q172168'), ('name', 'Galerius')]
[('sub', 'http://www.wikidata.org/entity/Q1429'), ('name', 'A

Final query for this task

In [100]:
# write your final query
queryString = """
SELECT DISTINCT ?killedBy ?sub ?emperorName ?killerName
WHERE {
   ?sub ?p wd:Q842606. #we get all the roman emperors.
   ?sub wdt:P157 ?killedBy. #we get the killers
   ?killedBy ?p wd:Q842606. #we indicate the killer is also a roman emperor
   # get the label
   ?killedBy sc:name ?killerName.
   ?sub sc:name ?emperorName.
   
}limit 20
"""

print("Results")
x=run_query(queryString)

Results
[('killedBy', 'http://www.wikidata.org/entity/Q8413'), ('sub', 'http://www.wikidata.org/entity/Q184549'), ('emperorName', 'Licinius'), ('killerName', 'Constantine the Great')]
[('killedBy', 'http://www.wikidata.org/entity/Q1446'), ('sub', 'http://www.wikidata.org/entity/Q183089'), ('emperorName', 'Geta'), ('killerName', 'Caracalla')]
2


## Task 6
Find the BGP for war

In [105]:
# write your queries
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   #wd:Q1048 ?p ?obj . #getting properties of "Julius Caesar"
   #wd:Q1048 wdt:P607 ?obj. #from "Julius Caesar"-"conflict" relation we get "gallic war"
   #wd:Q202161 ?p ?obj. #getting data of "gallic war"
   wd:Q202161 wdt:P31 ?obj. #from "gallic war"-"instance of" relation we get "war"
   
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q198'), ('name', 'war')]
1


Final query for this task

In [106]:
# write your final query
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q198 ?p ?obj.
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('name', 'Wikid

## Task 7
Find the wars (note that "war" and "battle" are not the same thing) participated by the Roman Empire (return IRI and name of the war).

In [132]:
# write your queries
queryString = """
SELECT DISTINCT ?wars ?name 
WHERE {
    
    ?wars wdt:P31 wd:Q198. #we find the all war names from "instance" relation through "war" data 
    ?wars wdt:P710 wd:Q2277. #we bind the each war in "participant" to "Roman Empire" 
    ?wars sc:name ?name.
}limit 20
"""

print("Results")
x=run_query(queryString)




Results
[('wars', 'http://www.wikidata.org/entity/Q6254'), ('name', 'Kitos War')]
[('wars', 'http://www.wikidata.org/entity/Q500409'), ('name', 'Roman–Persian Wars')]
[('wars', 'http://www.wikidata.org/entity/Q202578'), ('name', 'First Jewish-Roman War')]
[('wars', 'http://www.wikidata.org/entity/Q1533828'), ('name', 'Roman–Parthian War of 58–63')]
[('wars', 'http://www.wikidata.org/entity/Q3651751'), ('name', "Trajan's Parthian campaign")]
[('wars', 'http://www.wikidata.org/entity/Q933963'), ('name', 'Marcomannic Wars')]
[('wars', 'http://www.wikidata.org/entity/Q3651748'), ('name', 'Parthian war of Caracalla')]
[('wars', 'http://www.wikidata.org/entity/Q1790268'), ('name', 'Roman–Parthian War of 161–166')]
[('wars', 'http://www.wikidata.org/entity/Q84850006'), ('name', 'Roman–Parthian War of 195–198')]
9


Final query for this task

In [133]:
# write your final query
queryString = """
SELECT DISTINCT ?wars ?name 
WHERE {
    
    ?wars wdt:P31 wd:Q198. #we find the all war names from "instance" relation through "war" data 
    ?wars wdt:P710 wd:Q2277. #we bind the each war in "participant" to "Roman Empire" 
    ?wars sc:name ?name.
}limit 20
"""

print("Results")
x=run_query(queryString)




Results
[('wars', 'http://www.wikidata.org/entity/Q6254'), ('name', 'Kitos War')]
[('wars', 'http://www.wikidata.org/entity/Q500409'), ('name', 'Roman–Persian Wars')]
[('wars', 'http://www.wikidata.org/entity/Q202578'), ('name', 'First Jewish-Roman War')]
[('wars', 'http://www.wikidata.org/entity/Q1533828'), ('name', 'Roman–Parthian War of 58–63')]
[('wars', 'http://www.wikidata.org/entity/Q3651751'), ('name', "Trajan's Parthian campaign")]
[('wars', 'http://www.wikidata.org/entity/Q933963'), ('name', 'Marcomannic Wars')]
[('wars', 'http://www.wikidata.org/entity/Q3651748'), ('name', 'Parthian war of Caracalla')]
[('wars', 'http://www.wikidata.org/entity/Q1790268'), ('name', 'Roman–Parthian War of 161–166')]
[('wars', 'http://www.wikidata.org/entity/Q84850006'), ('name', 'Roman–Parthian War of 195–198')]
9


## Task 8
Consider all the wars participated by the Roman Empire. For each war return the number of related battles and the opponent (a war is composed of one or more battles) (the result set must be a list of 5-tuples war IRI, label, opponent IRI and label, and #battles).

In [163]:
# write your queries
queryString = """
SELECT DISTINCT ?participant ?wars ?opponent ?warname (count(?warParts) as ?numberOfBattles)
WHERE {
    
    ?wars wdt:P31 wd:Q198. #we find the all war names from "instance" relation through "war" data 
    ?wars wdt:P710 wd:Q2277. #we bind the each war in "participant" to "Roman Empire" 
    ?wars wdt:P710 ?participant.
    
    filter (?participant != wd:Q2277) #we eliminate "Roman Empire" because we are trying to find all the opponents which are versus "Roman Empire".

    optional{?wars wdt:P527 ?warParts}. #from wars we get the all the related wars by "parts of" relation. We add optional because some wars do not have related wars. Not to eliminate those wars we use optional.
    
    ?participant sc:name ?opponent.
    ?wars sc:name ?warname.
    
} group by ?wars ?warname ?participant ?opponent
limit 20
"""

print("Results")
x=run_query(queryString)




Results
[('participant', 'http://www.wikidata.org/entity/Q1986139'), ('wars', 'http://www.wikidata.org/entity/Q3651748'), ('opponent', 'Parthian Empire'), ('warname', 'Parthian war of Caracalla'), ('numberIfBattles', '0')]
[('participant', 'http://www.wikidata.org/entity/Q1986139'), ('wars', 'http://www.wikidata.org/entity/Q3651751'), ('opponent', 'Parthian Empire'), ('warname', "Trajan's Parthian campaign"), ('numberIfBattles', '0')]
[('participant', 'http://www.wikidata.org/entity/Q1986139'), ('wars', 'http://www.wikidata.org/entity/Q1790268'), ('opponent', 'Parthian Empire'), ('warname', 'Roman–Parthian War of 161–166'), ('numberIfBattles', '0')]
[('participant', 'http://www.wikidata.org/entity/Q12544'), ('wars', 'http://www.wikidata.org/entity/Q500409'), ('opponent', 'Byzantine Empire'), ('warname', 'Roman–Persian Wars'), ('numberIfBattles', '2')]
[('participant', 'http://www.wikidata.org/entity/Q17167'), ('wars', 'http://www.wikidata.org/entity/Q500409'), ('opponent', 'Roman Repub

Final query for this task

In [164]:
# write your queries
queryString = """
SELECT DISTINCT ?participant ?wars ?opponent ?warname (count(?warParts) as ?numberOfBattles)
WHERE {
    
    ?wars wdt:P31 wd:Q198. #we find the all war names from "instance" relation through "war" data 
    ?wars wdt:P710 wd:Q2277. #we bind the each war in "participant" to "Roman Empire" 
    ?wars wdt:P710 ?participant.
    
    filter (?participant != wd:Q2277) #we eliminate "Roman Empire" because we are trying to find all the opponents which are versus "Roman Empire".

    optional{?wars wdt:P527 ?warParts}. #from wars we get the all the related wars by "parts of" relation. We add optional because some wars do not have related wars. Not to eliminate those wars we use optional.
    
    ?participant sc:name ?opponent.
    ?wars sc:name ?warname.
    
} group by ?wars ?warname ?participant ?opponent
limit 20
"""

print("Results")
x=run_query(queryString)




Results
[('participant', 'http://www.wikidata.org/entity/Q1986139'), ('wars', 'http://www.wikidata.org/entity/Q3651748'), ('opponent', 'Parthian Empire'), ('warname', 'Parthian war of Caracalla'), ('numberOfBattles', '0')]
[('participant', 'http://www.wikidata.org/entity/Q1986139'), ('wars', 'http://www.wikidata.org/entity/Q3651751'), ('opponent', 'Parthian Empire'), ('warname', "Trajan's Parthian campaign"), ('numberOfBattles', '0')]
[('participant', 'http://www.wikidata.org/entity/Q1986139'), ('wars', 'http://www.wikidata.org/entity/Q1790268'), ('opponent', 'Parthian Empire'), ('warname', 'Roman–Parthian War of 161–166'), ('numberOfBattles', '0')]
[('participant', 'http://www.wikidata.org/entity/Q12544'), ('wars', 'http://www.wikidata.org/entity/Q500409'), ('opponent', 'Byzantine Empire'), ('warname', 'Roman–Persian Wars'), ('numberOfBattles', '2')]
[('participant', 'http://www.wikidata.org/entity/Q17167'), ('wars', 'http://www.wikidata.org/entity/Q500409'), ('opponent', 'Roman Repub